In [1]:
import pandas as pd
import re

import spacy
from spacy.language import Language
from spacy.tokens import Doc, Span


from spacy_language_detection import LanguageDetector
from googletrans import Translator


from langdetect import detect
import langid


In [2]:
df = pd.read_csv("../../business-climate/data/mergent_and_library/mergent_flagged.csv.xz")

In [3]:
#df = pd.read_csv("../../business-climate/data/mergent_and_library/final_data.csv.xz")

In [4]:
len(df)

166740

In [5]:
df = df[['duns', 'company_name0', 'address', 'city', 'state', 'zipcode']]

In [6]:
df.iloc[15]['company_name0']

'mistress nancy ava miller'

In [7]:
# spacy, google_translate, langid, textblob (N/A)


In [8]:
## Init Spacy: Run this only once!
import spacy
from spacy.language import Language
from spacy.tokens import Doc, Span


from spacy_language_detection import LanguageDetector


nlp_model = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=lambda nlp,name: LanguageDetector(seed=42)  )
nlp_model.add_pipe('language_detector', last=True)





In [9]:
white_languages = [
    'aa',  # Afar
    'af',  # Afrikaans
    'an',  # Aragonese
    'ay',  # Aymara
    'bi',  # Bislama
    'br',  # Breton
    'ca',  # Catalan
    'ce',  # Chechen
    'co',  # Corsican
    'cs',  # Czech
    'cy',  # Welsh
    'da',  # Danish
    'de',  # German
    'en',  # English
    'es',  # Spanish
    'et',  # Estonian
    'eu',  # Basque
    'fi',  # Finnish
    'fj',  # Fijian
    'fo',  # Faroese
    'fr',  # French
    'fy',  # Western Frisian
    'ga',  # Irish
    'gd',  # Scottish Gaelic
    'gl',  # Galician
    'gv',  # Manx
    'hr',  # Croatian
    'hu',  # Hungarian
    'is',  # Icelandic
    'it',  # Italian
    'lb',  # Luxembourgish
    'li',  # Limburgish
    'lt',  # Lithuanian
    'lv',  # Latvian
    'mg',  # Malagasy
    'mi',  # Māori
    'mk',  # Macedonian
    'ml',  # Malayalam
    'mr',  # Marathi
    'mt',  # Maltese
    'nds',  # Low German
    'nl',  # Dutch
    'nn',  # Norwegian Nynorsk
    'no',  # Norwegian
    'nr',  # South Ndebele
    'oc',  # Occitan
    'pl',  # Polish
    'pt',  # Portuguese
    'qu',  # Quechua
    'ro',  # Romanian
    'se',  # Northern Sami
    'sk',  # Slovak
    'sl',  # Slovenian
    'sq',  # Albanian
    'sv',  # Swedish
    'sw',  # Swahili
    'tk',  # Turkmen
    'tn',  # Tswana
    'tr',  # Turkish
    'ts',  # Tsonga
    've',  # Venda
    'vo',  # Volapük
    'wa',  # Walloon
    'xh',  # Xhosa
    'zu',  # Zulu
]


In [10]:
# Remove all non-alphabetic characters from the list and return words which are of min length 2
def get_clean_list(ls):
    return [elem for elem in ls.split(' ') if re.match(r'[a-zA-Z]{2,}', elem)]

In [11]:
get_clean_list('william j byrnes')

['william', 'byrnes']

In [12]:
def get_prob_spacy(name):
    doc = nlp_model(name)
    lang_d = doc._.language
    language = lang_d['language']
    score = lang_d['score']
    #print(lang_d)
    if language in white_languages:
        return 0
    else:
        return score


In [13]:
get_prob_spacy("clark solutions")

0

In [14]:
## For rough work

doc = nlp_model("Chang")
lang_d = doc._.language
language = lang_d['language']
score = lang_d['score']
print(lang_d)


{'language': 'tl', 'score': 0.8571382856348696}


In [15]:
## INIT googletrans

from googletrans import Translator
translator = Translator()


def get_prob_gt(text):
    language = translator.detect(text)
    #print(language)
    
    if type(language.lang) == list:
        if language.lang[0] in white_languages:
            return language.confidence[0]
        else:
            return 0
            
        
    if language.lang in white_languages:
        return 0
    else:
        return language.confidence 
    

In [16]:
get_prob_gt("William")

0

In [17]:
## Init langdetect
from langdetect import detect


def get_prob_ld(text):
    lang = detect(text)
    
    if lang in white_languages:
        return 0
    else:
        return 1

In [18]:
print(df.iloc[8257]['company_name0'])

the eyewear gallery


In [19]:
prob_arr = []

LEN = len(df)

list_spacy = []
list_gt = []
list_ld = []

for i in range(8257, 20000):
    if i%100 == 0:
        print(i)
        
    company_name = df.iloc[i]['company_name0']
    tokens = get_clean_list(company_name)
    
    prob_token_0 = []
    prob_0_fail = False
    prob_token_1 = []
    prob_1_fail = False
    prob_token_2 = []
    prob_2_fail = False
    
    try:
        prob_token_0 = list(map(get_prob_spacy, tokens))
    except:
        prob_0_fail = True
    
    try:
        prob_token_1 = list(map(get_prob_gt, tokens))
    except:
        prob_1_fail = True
    
    try:
        prob_token_2 = list(map(get_prob_ld, tokens))
    except:
        prob_2_fail = True
        
    
    if prob_0_fail or prob_1_fail or prob_2_fail:
        print("Failed to predict one of the things",prob_0_fail,prob_1_fail,prob_2_fail)
    
    count_spacy = 0
    count_gt = 0
    count_ld = 0
    
    prb_spacy = 0
    prb_gt = 0
    prb_ld = 0
    
    
    #print(prob_token_1)
    
    
    for prb_itr in range(len(prob_token_0)):
        if not prob_0_fail and prob_token_0[prb_itr] > 0:
            count_spacy += 1
            prb_spacy += prob_token_0[prb_itr]
        if not prob_1_fail and prob_token_1[prb_itr] > 0:
            count_gt += 1
            prb_gt += prob_token_1[prb_itr]
        if not prob_2_fail and prob_token_2[prb_itr] > 0:
            count_ld += 1
            prb_ld += prob_token_2[prb_itr]
        
    #print(count_spacy,count_gt,count_ld )
    #print(prb_spacy, prb_gt, prb_ld)
            
    if not prob_0_fail:
        if count_spacy > 0:   
            prb_spacy = prb_spacy/count_spacy
        else:
            prb_spacy = 0
    else:
        prb_spacy = None
        
    if not prob_1_fail:
        if count_gt > 0:
            prb_gt = prb_gt/count_gt
        else:
            prb_gt = 0
    else:
        prb_gt = None
        
    if not prob_2_fail:
        if count_ld > 0:
            prb_ld = prb_ld/count_ld
        else:
            prb_ld = 0
    else:
        prb_ld = None
        
    list_spacy.append(prb_spacy)
    list_gt.append(prb_gt)
    list_ld.append(prb_ld)
    
    

    




    

8300
8400
8500
8600
8700
8800
8900
9000
9100
Failed to predict one of the things False True False
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18500
18600
18700
18800
18900
19000
19100
19200
19300
19400
19500
19600
19700
19800
19900


[None, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [20]:
len(list_ld)

11743

In [22]:

    
df_spacy = pd.DataFrame(list_spacy, columns=['prob_spacy'])
df_gt = pd.DataFrame(list_gt, columns=['prob_gt'])
df_ld = pd.DataFrame(list_ld, columns=['prob_ld'])


#df = df[:10001]

df_final = pd.concat([df_spacy, df_gt, df_ld], axis=1)


df_final.to_csv("training_set_8257_20000.csv")



10518

['guston', 'mart', 'deli']

True

In [43]:
df1 = pd.read_csv("./data/training/mergent_flagged_company_name_nlp_0_10000.csv")[:10000]
df2 = pd.read_csv("./data/training/mergent_flagged_company_name_nlp_10000_EOL.csv")[9990:]

In [35]:
df_final = pd.concat([df1, df2], axis=0)

In [44]:
df2.head()

,Unnamed: 0,duns,company_name,firstname,lastname,title,gender,flag_owner,flag_presi_vicepresi,prob_nw_eth_wiki,...,primary_naics,naics2,naics_name,flag_executive_reported,geoid,prop_nwh,geometry,prob_spacy,prob_gt,prob_ld
9990,9990,08-836-5767,CARAHSOFT TECHNOLOGY CORP.,Tess,Van Gorder,Partner Webcast Email Banner,FEMALE,0,0,0.049304,...,541512,54,"Professional, Scientific, and Technical Services",1,5.105948e+10,27.077295,"c(-77.344876, 38.950388)",NaN,NaN,NaN
9991,9991,08-836-5767,CARAHSOFT TECHNOLOGY CORP.,Erin,Portare,Account Representative,FEMALE,0,0,0.024183,...,541512,54,"Professional, Scientific, and Technical Services",1,5.105948e+10,27.077295,"c(-77.344876, 38.950388)",NaN,NaN,NaN
9992,9992,08-836-5767,CARAHSOFT TECHNOLOGY CORP.,Brian,Soliday,President,MALE,0,1,0.067371,...,541512,54,"Professional, Scientific, and Technical Services",1,5.105948e+10,27.077295,"c(-77.344876, 38.950388)",NaN,NaN,NaN
9993,9993,08-836-5767,CARAHSOFT TECHNOLOGY CORP.,Kate,Bojanowski,Order Management Specialist,FEMALE,0,0,0.032427,...,541512,54,"Professional, Scientific, and Technical Services",1,5.105948e+10,27.077295,"c(-77.344876, 38.950388)",NaN,NaN,NaN
9994,9994,08-836-5767,CARAHSOFT TECHNOLOGY CORP.,Amira,Ali,Accounts Receivable,FEMALE,0,0,0.330501,...,541512,54,"Professional, Scientific, and Technical Services",1,5.105948e+10,27.077295,"c(-77.344876, 38.950388)",NaN,NaN,NaN


In [37]:
df_final.to_csv("final_data.csv")